### 4.2 模型参数的访问、初始化和共享

In [1]:
import torch
from torch import nn
from torch.nn import init

In [5]:
net = nn.Sequential(
    nn.Linear(4, 3), # pytorch默认初始化
    nn.ReLU(),
    nn.Linear(3, 1)
)

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()
print(Y.item())

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
-0.5975306034088135


### 4.2.1 访问模型参数

> 1. **对于`Sequential`实例中含模型参数的层,可通过`Module`类的`parameters()`或者 `named_parameters()` 方法来访问所有参数(以迭代器形式)**
> 2. **`named_parameters()` 返回的名字自动加上了索引作为前缀**
> 3. **通过`net[index]`来访问网络任一层，此时返回的name没有索引前缀**
> 4. **param的类型`torch.nn.parameter.Parameter`,是`Tensor`的子类,如果一个`Tensor`是`Parameter`,那么会自动被添加到模型参数列表**
> 5. **`Parameter`是`Tensor`.可根据`data`访问参数值,`grad`访问梯度**

In [8]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [10]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [11]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        # weight1 是Parameter类型，会添加到网络参数中
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        # 而weight2却不会
        self.weight2 = torch.rand(20, 20)
    
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [14]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[ 0.3011,  0.4198,  0.3230,  0.2946],
        [ 0.2989,  0.3115, -0.3921,  0.2833],
        [ 0.2363,  0.0392,  0.2317, -0.2515]])
None
tensor([[ 0.6614,  0.4827,  0.4717,  0.2036],
        [-0.7584, -0.5535, -0.5410, -0.2335],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])


### 4.2.2 自定义初始化方法

In [17]:
# 使用init模块，初始化权重为N(0, 0.01)
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 7.0761e-05, -2.0239e-03, -1.0443e-02,  7.8113e-05],
        [ 1.0639e-02,  4.1825e-03, -5.3804e-03, -5.0294e-03],
        [ 5.2995e-03, -3.6706e-03, -2.2803e-02,  6.4969e-04]])
2.weight tensor([[ 0.0117, -0.0019,  0.0093]])


In [18]:
# 使用init模块，初始化偏置为 0
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 4.2.3 自定义初始化方法

> pytorch 源码
```python
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)
```

In [24]:
# 权重一半概率初始化为0，另一半概率初始化为[-15, -5]和[5, 10]的两个区间均匀分布
def init_weight_(tensor):
    with torch.no_grad():
        # 均匀分布
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

In [25]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[ 7.8997,  0.0000,  7.5120, -9.1512],
        [-7.8311, -0.0000,  8.1823, -0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.0000]])
2.weight tensor([[-0., 0., 0.]])


In [23]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([2., 2., 2.])
2.bias tensor([2.])


### 4.2.4 共享模型参数

In [26]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [27]:
# 这两个线性层是一个对象
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [29]:
# 模型参数里包含了梯度,所以反向传播是，这些共享参数的梯度是累加的
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) # 单次是3，所以两次是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
